In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.constants import c

In [2]:
![./phased_array_intro.png](Phased Array image)

/bin/bash: -c: line 0: syntax error near unexpected token `Phased'
/bin/bash: -c: line 0: `[./phased_array_intro.png](Phased Array image)'


In [3]:
def iso_free_path_loss(lam, r):
    return (lam/(4*np.pi*r))**2

def isotropic_characteristic(p, theta, phi):
    return 1

In [ ]:
def heatmap(data, xs, ys, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(xs)
    ax.set_yticks(ys)

    # Let the horizontal axes labeling appear on top.
#     ax.tick_params(top=True, bottom=False,
#                    labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
#     plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
#              rotation_mode="anchor")

    # Turn spines off and create white grid.
#     ax.spines[:].set_visible(False)

#     ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
#     ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
#     ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
#     ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


In [ ]:
def single_antenna_power_2d(params,x,y):
    lam = params[0]
    r = np.sqrt(x**2+y**2)
    phi = np.arctan2(y,x)
    theta = np.pi/2
    pout = iso_free_path_loss(lam, r) * isotropic_characteristic(params, theta, phi)
    return 10*np.log10(pout)

xs = np.arange(-0.5,0.5,0.01)
ys = np.arange(-0.5,0.5,0.01)
xxs, yys = np.meshgrid(xs, ys)
data = single_antenna_power_2d([0.003],xxs,yys)

heatmap(data, xs, ys)